# Tutorial 02: IRF Analysis -- Solutions

This notebook contains **complete solutions** for all exercises in Tutorial 02 (Impulse Response Function Analysis).

Each exercise is presented with its original description followed by a fully worked solution including code, output interpretation, and discussion.

---

**Exercises covered:**

| Exercise | Topic | Difficulty |
|----------|-------|------------|
| 1 | Ordering Sensitivity | Medium |
| 2 | Bootstrap CI Analysis | Easy |
| 3 | Peak Effect Identification | Medium |
| 4 | Multiplier Calculation | Hard |

---

## Setup

In [ ]:
import sys
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
np.random.seed(42)
warnings.filterwarnings('ignore')

project_root = Path('../../../').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

sys.path.insert(0, '../utils')

from panelbox.var import PanelVARData, PanelVAR

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams.update({'figure.figsize': (10, 6), 'figure.dpi': 100, 'font.size': 11})

### Load Data and Estimate Model

In [ ]:
df = pd.read_csv('../data/macro_panel.csv')

# Convert quarter strings to pandas PeriodIndex for proper temporal handling
df['quarter'] = pd.PeriodIndex(df['quarter'], freq='Q')

endog_vars = ['gdp_growth', 'inflation', 'interest_rate']
data = PanelVARData(df, endog_vars=endog_vars, entity_col='country', time_col='quarter', lags=2)
model = PanelVAR(data)
results = model.fit(method='ols', cov_type='clustered')
print(f"Model estimated: {data.endog_vars}, lags={data.p}")
print(f"Entities (N): {data.N}, Variables (K): {data.K}")
print(f"Observations: {results.n_obs}")
print(f"Stability: {'STABLE' if results.is_stable() else 'UNSTABLE'}")

---

## Exercise 1: Ordering Sensitivity (Medium)

Using the macro panel data:

1. Compute Cholesky IRFs with at least **3 different orderings** of the variables `['interest_rate', 'inflation', 'gdp_growth']`
2. Plot the **GDP response to an interest rate shock** for all 3 orderings on the same chart
3. Compare the results and discuss: How sensitive is this particular IRF to the ordering?
4. Create a table of peak effects and horizons
5. Discuss which ordering is most plausible economically

In [ ]:
# Exercise 1 Solution: Define 3 different Cholesky orderings
# Each ordering reflects a different assumption about contemporaneous causality

orderings = {
    'Policy First':    ['interest_rate', 'inflation', 'gdp_growth'],
    'Real Sector First': ['gdp_growth', 'inflation', 'interest_rate'],
    'Prices First':    ['inflation', 'gdp_growth', 'interest_rate'],
}

# Compute Cholesky IRFs for each ordering
irf_results = {}
for label, order in orderings.items():
    irf_results[label] = results.irf(
        periods=20,
        method='cholesky',
        order=order,
    )
    print(f"Computed IRF with ordering '{label}': {order}")

In [ ]:
# Plot GDP response to interest_rate shock for all 3 orderings
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

colors = ['#2166ac', '#b2182b', '#4dac26']
styles = ['-', '--', '-.']
horizons = np.arange(21)

for idx, (label, irf_obj) in enumerate(irf_results.items()):
    ax = axes[idx]
    gdp_response = irf_obj['gdp_growth', 'interest_rate']
    
    ax.plot(horizons, gdp_response, color=colors[idx], linewidth=2.2,
            marker='o', markersize=3, label=label)
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.fill_between(horizons, 0, gdp_response, alpha=0.15, color=colors[idx])
    
    # Mark peak
    peak_h = np.argmax(np.abs(gdp_response))
    peak_val = gdp_response[peak_h]
    ax.annotate(
        f'Peak: {peak_val:.4f}\nh={peak_h}',
        xy=(peak_h, peak_val),
        xytext=(peak_h + 3, peak_val + 0.01 * np.sign(peak_val)),
        fontsize=9,
        arrowprops=dict(arrowstyle='->', color='gray'),
        bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', edgecolor='gray'),
    )
    
    order = orderings[label]
    ax.set_title(f'{label}\n{" -> ".join(order)}', fontsize=11, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=11)
    if idx == 0:
        ax.set_ylabel('Response of GDP Growth', fontsize=11)
    ax.grid(True, alpha=0.3)

fig.suptitle('GDP Growth Response to Interest Rate Shock\nacross Cholesky Orderings',
             fontsize=14, fontweight='bold', y=1.05)
fig.tight_layout()
plt.show()

In [ ]:
# Overlay all 3 orderings on a single chart for direct comparison
fig, ax = plt.subplots(figsize=(10, 6))

for idx, (label, irf_obj) in enumerate(irf_results.items()):
    gdp_response = irf_obj['gdp_growth', 'interest_rate']
    ax.plot(horizons, gdp_response, color=colors[idx], linewidth=2.2,
            linestyle=styles[idx], label=label, marker='o', markersize=3)

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel('Response of GDP Growth', fontsize=12)
ax.set_title('Ordering Sensitivity: GDP Growth Response to Interest Rate Shock',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

In [ ]:
# Create summary table of peak effects and horizons
rows = []
for label, irf_obj in irf_results.items():
    gdp_response = irf_obj['gdp_growth', 'interest_rate']
    peak_h = int(np.argmax(np.abs(gdp_response)))
    peak_val = gdp_response[peak_h]
    impact = gdp_response[0]
    long_run = gdp_response[-1]
    sign = 'Positive' if peak_val > 0 else 'Negative'
    
    rows.append({
        'Ordering': label,
        'Impact (h=0)': f'{impact:.6f}',
        'Peak Horizon': peak_h,
        'Peak Magnitude': f'{peak_val:.6f}',
        'Sign': sign,
        'Long-run (h=20)': f'{long_run:.6f}',
    })

df_summary = pd.DataFrame(rows)
print('\n=== Peak Effect Summary Across Orderings ===')
print('=' * 80)
print(df_summary.to_string(index=False))

# Compute max difference across orderings at each horizon
all_responses = np.column_stack([
    irf_results[label]['gdp_growth', 'interest_rate']
    for label in irf_results
])
max_diff_by_h = np.max(all_responses, axis=1) - np.min(all_responses, axis=1)

print(f'\nMax difference across orderings at each horizon:')
print(f'  Max absolute difference: {np.max(max_diff_by_h):.6f} (at h={np.argmax(max_diff_by_h)})')
print(f'  Average difference:      {np.mean(max_diff_by_h):.6f}')

### Discussion: Ordering Sensitivity

**Key findings:**

1. **Policy First** ordering (`interest_rate -> inflation -> gdp_growth`) is the most standard in the monetary policy VAR literature (Christiano, Eichenbaum, Evans, 1999). It assumes the central bank sets rates based on last period's information, so interest rates are predetermined within the quarter.

2. **Real Sector First** ordering (`gdp_growth -> inflation -> interest_rate`) assumes real output is most exogenous and that monetary policy reacts to both GDP and inflation contemporaneously. This reflects a Taylor rule perspective.

3. **Prices First** ordering (`inflation -> gdp_growth -> interest_rate`) assumes price signals are the fastest to propagate.

**Which is most plausible?** The "Policy First" ordering has the strongest theoretical justification:
- Central banks typically make policy decisions at discrete intervals (meetings)
- Interest rate decisions are based on information available at the time of the meeting (lagged data)
- This makes the interest rate predetermined relative to within-quarter movements in GDP and inflation

**Sensitivity:** If the responses are qualitatively similar across orderings (same sign, similar timing), the results are robust. If they differ substantially, the identification is fragile, and Generalized IRFs may be preferred.

---

## Exercise 2: Bootstrap CI Analysis (Easy)

Using the macro panel VAR:

1. Compute bootstrap IRFs with `n_bootstrap=200` and `ci_level=0.90` (instead of 0.95)
2. Also compute with `ci_level=0.95`
3. Plot both on the same axes
4. Table of CI widths at selected horizons

In [ ]:
# Exercise 2 Solution: Compute IRFs with 90% and 95% bootstrap CIs

# 90% confidence interval
irf_90 = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.90,
    seed=42,
    verbose=False,
)

# 95% confidence interval
irf_95 = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

print(f'90% CI computed: ci_lower shape = {irf_90.ci_lower.shape}')
print(f'95% CI computed: ci_upper shape = {irf_95.ci_upper.shape}')

In [ ]:
# Plot GDP response to interest rate shock with both 90% and 95% CIs
response_var = 'gdp_growth'
impulse_var = 'interest_rate'

# Point estimate (same for both since same method/ordering)
irf_values = irf_95[response_var, impulse_var]
horizons = np.arange(len(irf_values))

# Extract CI bounds using index positions
resp_idx = irf_95.var_names.index(response_var)
imp_idx = irf_95.var_names.index(impulse_var)

lower_95 = irf_95.ci_lower[:, resp_idx, imp_idx]
upper_95 = irf_95.ci_upper[:, resp_idx, imp_idx]
lower_90 = irf_90.ci_lower[:, resp_idx, imp_idx]
upper_90 = irf_90.ci_upper[:, resp_idx, imp_idx]

fig, ax = plt.subplots(figsize=(12, 6))

# 95% CI (wider, lighter)
ax.fill_between(horizons, lower_95, upper_95,
                alpha=0.15, color='#2166ac', label='95% Bootstrap CI')
ax.plot(horizons, lower_95, color='#2166ac', linewidth=0.7, linestyle=':', alpha=0.5)
ax.plot(horizons, upper_95, color='#2166ac', linewidth=0.7, linestyle=':', alpha=0.5)

# 90% CI (narrower, darker)
ax.fill_between(horizons, lower_90, upper_90,
                alpha=0.25, color='#b2182b', label='90% Bootstrap CI')
ax.plot(horizons, lower_90, color='#b2182b', linewidth=0.7, linestyle='--', alpha=0.5)
ax.plot(horizons, upper_90, color='#b2182b', linewidth=0.7, linestyle='--', alpha=0.5)

# Point estimate
ax.plot(horizons, irf_values, color='black', linewidth=2.2,
        label='Point Estimate', marker='o', markersize=3)

# Zero line
ax.axhline(y=0, color='gray', linewidth=0.8, linestyle='--', alpha=0.5)

ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel(f'Response of {response_var}', fontsize=12)
ax.set_title(f'{response_var} Response to {impulse_var} Shock\n90% vs 95% Bootstrap Confidence Intervals (B=200)',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

In [ ]:
# Table of CI widths at selected horizons
selected_horizons = [0, 1, 2, 4, 8, 12, 16, 20]

rows = []
for h in selected_horizons:
    width_90 = upper_90[h] - lower_90[h]
    width_95 = upper_95[h] - lower_95[h]
    
    # Check significance
    sig_90 = (upper_90[h] < 0) or (lower_90[h] > 0)
    sig_95 = (upper_95[h] < 0) or (lower_95[h] > 0)
    
    rows.append({
        'Horizon': h,
        'Point Est.': f'{irf_values[h]:.4f}',
        '90% CI Width': f'{width_90:.4f}',
        '95% CI Width': f'{width_95:.4f}',
        'Ratio (95/90)': f'{width_95 / width_90:.3f}' if width_90 > 0 else 'N/A',
        'Sig. 90%': 'YES' if sig_90 else 'no',
        'Sig. 95%': 'YES' if sig_95 else 'no',
    })

df_ci = pd.DataFrame(rows)
print('=== CI Width Comparison: 90% vs 95% ===')
print('=' * 80)
print(df_ci.to_string(index=False))

print('\n--- Interpretation ---')
print('The 95% CI is wider than the 90% CI at every horizon.')
print('If a response is significant at 95%, it is automatically significant at 90%.')
print('The 90% CI may reveal additional horizons where the response is significant.')

### Discussion: Bootstrap CI Analysis

**Key findings:**

1. The 95% CI is always wider than the 90% CI (as expected -- higher confidence requires a wider interval).
2. The ratio of widths (95% / 90%) is approximately constant across horizons, reflecting the distributional properties of the bootstrap.
3. Changing from 95% to 90% confidence may reveal additional horizons where the response is statistically significant (the CI excludes zero).
4. CI widths generally increase with the horizon, reflecting the accumulation of estimation uncertainty over time.

**Practical guidance:** Use 95% CIs for standard inference and 90% CIs for a less conservative assessment. Always report the confidence level used.

---

## Exercise 3: Peak Effect Identification (Medium)

For all 9 impulse-response pairs (3 shocks x 3 responses) in the macro VAR:

1. Compute Cholesky IRFs with the standard ordering `['interest_rate', 'inflation', 'gdp_growth']`
2. For each pair, identify the horizon of the peak (absolute) effect, the magnitude, and the sign
3. Present the results in a summary table (pandas DataFrame)
4. Which shock has the largest absolute peak effect on GDP growth? At what horizon?

In [ ]:
# Exercise 3 Solution: Peak effect identification for all 9 pairs

# Compute Cholesky IRF with standard ordering
irf_standard = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
)

var_names = irf_standard.var_names
print(f'Variables: {var_names}')
print(f'Number of IRF pairs: {len(var_names) ** 2}')

In [ ]:
# Loop over all impulse-response pairs and identify peak effects
rows = []

for impulse_var in var_names:
    for response_var in var_names:
        irf_vals = irf_standard[response_var, impulse_var]
        
        # Find peak absolute effect
        peak_h = int(np.argmax(np.abs(irf_vals)))
        peak_val = irf_vals[peak_h]
        sign = 'Positive' if peak_val > 0 else 'Negative'
        
        # Also compute impact and long-run
        impact = irf_vals[0]
        long_run = irf_vals[-1]
        
        rows.append({
            'Impulse': impulse_var,
            'Response': response_var,
            'Peak Horizon': peak_h,
            'Peak Magnitude': peak_val,
            'Abs. Peak': abs(peak_val),
            'Sign': sign,
            'Impact (h=0)': impact,
            'Long-run (h=20)': long_run,
        })

df_peaks = pd.DataFrame(rows)

# Display formatted table
print('=== Peak Effect Summary for All Impulse-Response Pairs ===')
print('=' * 95)
df_display = df_peaks[['Impulse', 'Response', 'Peak Horizon', 'Peak Magnitude', 'Sign', 'Impact (h=0)', 'Long-run (h=20)']].copy()
df_display['Peak Magnitude'] = df_display['Peak Magnitude'].apply(lambda x: f'{x:.6f}')
df_display['Impact (h=0)'] = df_display['Impact (h=0)'].apply(lambda x: f'{x:.6f}')
df_display['Long-run (h=20)'] = df_display['Long-run (h=20)'].apply(lambda x: f'{x:.6f}')
print(df_display.to_string(index=False))

In [ ]:
# Identify the largest absolute peak effect on GDP growth
gdp_rows = df_peaks[df_peaks['Response'] == 'gdp_growth'].copy()
gdp_rows_sorted = gdp_rows.sort_values('Abs. Peak', ascending=False)

print('\n=== Shocks Ranked by Absolute Peak Effect on GDP Growth ===')
print('=' * 70)
for _, row in gdp_rows_sorted.iterrows():
    print(f"  {row['Impulse']:>15s} shock -> GDP Growth: "
          f"peak = {row['Peak Magnitude']:.6f} at h={row['Peak Horizon']} ({row['Sign']})")

top = gdp_rows_sorted.iloc[0]
print(f"\n>>> Largest absolute peak on GDP growth: "
      f"{top['Impulse']} shock with magnitude {top['Peak Magnitude']:.6f} at h={top['Peak Horizon']}")

In [ ]:
# Heatmap visualization of peak magnitudes
pivot_peak = df_peaks.pivot(index='Response', columns='Impulse', values='Peak Magnitude')
pivot_horizon = df_peaks.pivot(index='Response', columns='Impulse', values='Peak Horizon')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Peak magnitude heatmap
sns.heatmap(pivot_peak, annot=True, fmt='.4f', cmap='RdBu_r', center=0,
            linewidths=0.5, ax=axes[0], cbar_kws={'label': 'Magnitude'})
axes[0].set_title('Peak IRF Magnitude', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Impulse Variable', fontsize=11)
axes[0].set_ylabel('Response Variable', fontsize=11)

# Peak horizon heatmap
sns.heatmap(pivot_horizon, annot=True, fmt='.0f', cmap='YlOrRd',
            linewidths=0.5, ax=axes[1], cbar_kws={'label': 'Horizon'})
axes[1].set_title('Horizon of Peak Effect', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Impulse Variable', fontsize=11)
axes[1].set_ylabel('Response Variable', fontsize=11)

fig.suptitle('Peak Effect Analysis: All Impulse-Response Pairs',
             fontsize=14, fontweight='bold', y=1.03)
fig.tight_layout()
plt.show()

### Discussion: Peak Effect Identification

**Summary of findings:**

- Own shocks (diagonal elements) typically have the largest peak effects and occur at h=0 (impact).
- Cross-variable effects tend to peak at later horizons (h > 0), reflecting transmission lags.
- The peak horizon for GDP growth's response to interest rate shocks reflects the monetary policy transmission lag.
- Negative cross-effects (e.g., interest rate shock reducing GDP growth) are consistent with standard macroeconomic theory.

---

## Exercise 4: Multiplier Calculation (Hard)

Using both level and cumulative IRFs from the macro VAR:

1. Compute the short-run multiplier (h=4), medium-run (h=8), and long-run (h=20) multipliers
2. Verify the cumulative relationship: `cumulative_irf[h] = sum(level_irf[0:h+1])`
3. Create a formatted multiplier table
4. Plot a "multiplier timeline" showing the evolution of the cumulative multiplier

In [ ]:
# Exercise 4 Solution: Compute level and cumulative IRFs

cholesky_order = ['interest_rate', 'inflation', 'gdp_growth']

# Level IRF
irf_level = results.irf(
    periods=20,
    method='cholesky',
    order=cholesky_order,
    cumulative=False,
)

# Cumulative IRF
irf_cumulative = results.irf(
    periods=20,
    method='cholesky',
    order=cholesky_order,
    cumulative=True,
)

print(f'Level IRF computed: cumulative={irf_level.cumulative}')
print(f'Cumulative IRF computed: cumulative={irf_cumulative.cumulative}')

In [ ]:
# Extract multipliers for all variables in response to interest rate shock
impulse = 'interest_rate'
response_vars = irf_level.var_names
multiplier_horizons = {'Short-run (h=4)': 4, 'Medium-run (h=8)': 8, 'Long-run (h=20)': 20}

print('=== Dynamic Multipliers: Response to Interest Rate Shock ===')
print('=' * 75)

rows = []
for resp_var in response_vars:
    cum_vals = irf_cumulative[resp_var, impulse]
    level_vals = irf_level[resp_var, impulse]
    
    row = {'Response Variable': resp_var}
    for label, h in multiplier_horizons.items():
        row[label] = cum_vals[h]
    rows.append(row)

df_multipliers = pd.DataFrame(rows)
print(df_multipliers.to_string(index=False, float_format='%.6f'))

In [ ]:
# Verify: cumulative_irf[h] == sum(level_irf[0:h+1]) for all horizons
print('=== Verification: Cumulative IRF = Sum of Level IRFs ===')
print('=' * 70)

max_errors = []
for resp_var in response_vars:
    level_vals = irf_level[resp_var, impulse]
    cum_vals = irf_cumulative[resp_var, impulse]
    
    # Compute manual cumulative sum
    manual_cumsum = np.cumsum(level_vals)
    
    # Check difference
    max_err = np.max(np.abs(cum_vals - manual_cumsum))
    max_errors.append(max_err)
    
    print(f'\n  {resp_var} <- {impulse}:')
    print(f'    Max absolute error: {max_err:.2e}')
    
    # Show a few horizons for detailed verification
    for h in [4, 8, 20]:
        print(f'    h={h:2d}: cumulative_irf={cum_vals[h]:.8f}, '
              f'sum(level[0:{h+1}])={manual_cumsum[h]:.8f}, '
              f'diff={abs(cum_vals[h] - manual_cumsum[h]):.2e}')

print(f'\n>>> All verifications passed: max error = {max(max_errors):.2e}')
print('    The cumulative IRF is exactly the running sum of level IRFs.')

In [ ]:
# Detailed multiplier table with all horizons
print('\n=== Complete Multiplier Table: Interest Rate Shock ===')
print('=' * 80)

print(f'{"Horizon":>8} {"Type":>14}', end='')
for var in response_vars:
    print(f' {var:>16}', end='')
print()
print('-' * 80)

for h in [0, 1, 2, 4, 8, 12, 16, 20]:
    # Level (period-by-period)
    print(f'{h:>8} {"Level":>14}', end='')
    for var in response_vars:
        val = irf_level[var, impulse][h]
        print(f' {val:>16.6f}', end='')
    print()
    
    # Cumulative (multiplier)
    print(f'{"":>8} {"Cumulative":>14}', end='')
    for var in response_vars:
        val = irf_cumulative[var, impulse][h]
        print(f' {val:>16.6f}', end='')
    print()
    print()

In [ ]:
# Plot: Multiplier timeline -- cumulative IRF evolution from h=0 to h=20
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
horizons = np.arange(21)
colors_resp = ['#2166ac', '#d6604d', '#4dac26']

for idx, resp_var in enumerate(response_vars):
    ax = axes[idx]
    
    # Level IRF (bars)
    level_vals = irf_level[resp_var, impulse]
    ax.bar(horizons, level_vals, alpha=0.3, color=colors_resp[idx],
           label='Level IRF (period effect)', width=0.8)
    
    # Cumulative IRF (line)
    cum_vals = irf_cumulative[resp_var, impulse]
    ax.plot(horizons, cum_vals, color=colors_resp[idx], linewidth=2.5,
            marker='s', markersize=4, label='Cumulative IRF (multiplier)')
    
    # Annotate key multiplier horizons
    for h_label, h_val in [(4, 4), (8, 8), (20, 20)]:
        ax.annotate(
            f'h={h_val}: {cum_vals[h_val]:.4f}',
            xy=(h_val, cum_vals[h_val]),
            xytext=(h_val + 1.5, cum_vals[h_val] + 0.02 * np.sign(cum_vals[h_val]) if cum_vals[h_val] != 0 else 0.02),
            fontsize=8,
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.8),
            bbox=dict(boxstyle='round,pad=0.2', facecolor='lightyellow', edgecolor='gray', alpha=0.8),
        )
    
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'Response: {resp_var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=11)
    if idx == 0:
        ax.set_ylabel('IRF Value', fontsize=11)
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.3)

fig.suptitle('Multiplier Timeline: Cumulative Response to Interest Rate Shock',
             fontsize=14, fontweight='bold', y=1.03)
fig.tight_layout()
plt.show()

In [ ]:
# Summary: Short-run, medium-run, and long-run multipliers as a clean table
print('\n=== Final Multiplier Summary ===')
print('=' * 65)
print(f'{"Response":>18} {"Short-run (h=4)":>16} {"Medium-run (h=8)":>17} {"Long-run (h=20)":>16}')
print('-' * 65)

for resp_var in response_vars:
    cum_vals = irf_cumulative[resp_var, impulse]
    print(f'{resp_var:>18} {cum_vals[4]:>16.6f} {cum_vals[8]:>17.6f} {cum_vals[20]:>16.6f}')

print('\n--- Economic Interpretation ---')
print('Since variables are in growth rates (percentage points per quarter):')
print('  - The LEVEL IRF at horizon h gives the effect on the growth rate at quarter h.')
print('  - The CUMULATIVE IRF at horizon h gives the total accumulated effect on the')
print('    level of the variable from period 0 through h.')
print('  - For GDP growth: a cumulative multiplier of X at h=8 means that GDP is')
print('    X percentage points higher (or lower) after 2 years compared to the')
print('    no-shock scenario.')
print('  - The long-run multiplier (h=20) approximates the permanent level shift.')

### Discussion: Multiplier Calculation

**Key findings:**

1. **Cumulative verification**: The cumulative IRF at each horizon exactly equals the sum of level IRFs from 0 to h, confirming the mathematical relationship.

2. **Short-run multiplier (h=4)**: Captures the first-year accumulated effect. For GDP growth, this shows how much output is affected in the first year after a monetary policy shock.

3. **Medium-run multiplier (h=8)**: The two-year accumulated effect. This is often the most policy-relevant horizon for monetary policy analysis.

4. **Long-run multiplier (h=20)**: Approximates the total permanent effect. In a stationary VAR, this converges as h increases.

5. **Economic significance**: If the cumulative GDP response to a 1 s.d. interest rate shock is, say, -0.15 at h=8, this means that the **level** of GDP is 0.15 percentage points lower after 2 years. For a country with 2% trend growth, this represents roughly a 7.5% reduction relative to the growth rate, which is economically significant.

---

## End of Solutions

These solutions demonstrate the key techniques for IRF analysis:
- Testing robustness to Cholesky ordering
- Comparing bootstrap CI widths across confidence levels
- Systematically identifying peak effects across all variable pairs
- Computing and verifying dynamic multipliers from cumulative IRFs